In [1]:
import pandas as pd
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
urlav = "https://raw.githubusercontent.com/olagit/Structure-vs-properties-of-compounds/main/data/maccs.csv"
maccs = pd.read_csv(urlav)

In [3]:
maccs = maccs.drop(["index", "smiles"], axis=1)
maccs

,logP,fp0,fp1,fp2,fp3,fp4,fp5,fp6,fp7,fp8,...,fp157,fp158,fp159,fp160,fp161,fp162,fp163,fp164,fp165,fp166
0,-0.770,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,1,1,1,1,0
1,-3.300,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
2,-2.060,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,-7.870,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
4,-1.330,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,-1.710,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1124,0.106,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,1,0,0
1125,-3.091,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
1126,-3.180,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [4]:
X = maccs.iloc[:,1:]
y = maccs.iloc[:,0]
X = torch.tensor(X.values)
Y = torch.tensor(y.values)
X.shape

torch.Size([1128, 167])

In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from typing import List, Tuple

from mldd.metrics import mae, rmse, rocauc, r_squared
from mldd.data import *

def train(X_train, y_train, X_valid, y_valid):
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001

    model = nn.Sequential(nn.ReLU(),
                          nn.Softmax(dim=1),
                          nn.Linear(167, 1))
                          
    model.train()

    dataset = TensorDataset(torch.FloatTensor(X_train.float()), torch.FloatTensor(y_train.float().reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    loss_fn = nn.MSELoss()
    for epoch in trange(1, epochs + 1, leave=False):
        for X, y in tqdm(loader, leave=False):
            model.zero_grad()
            preds = model(X)
            loss = loss_fn(preds,y)
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    batch_size = 64

    dataset = TensorDataset(torch.FloatTensor(X_test.float()), torch.FloatTensor(y_test.float().reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    preds_batches = []
    with torch.no_grad():
        for X, Y in tqdm(loader):
            preds = model(X)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.5, random_state=123)
print(torch.FloatTensor(y_test.float()))
print(y_test.type())

scores = []

for _ in range(X_train.shape[0]):

    model = train(X_train, y_train, X_valid, y_valid)
    
    predictions = predict(model, X_test, y_test)
    
    rmse_score = np.sqrt(np.power(y_test[:,None] - predictions, 2).mean())
    mae_score = np.abs(y_test[:,None] - predictions).mean()
    scores.append([rmse_score, mae_score])
    
#     break 

scores = np.array(scores)
print('RMSE, MAE = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))

print(predictions)


tensor([ -4.5540,  -1.7900,  -7.0100,  -5.1600,  -0.9200,  -1.6000,  -0.5100,
         -4.6780,  -3.7850,  -8.0030,  -3.0100,  -1.8400,  -2.3300,   1.0200,
         -1.9500,  -3.7700,  -1.7700,  -0.4000,  -1.9500,  -0.5000,  -0.6300,
         -2.8430,  -6.1400,  -3.0800,   0.3200,  -6.8760,  -5.2400,  -8.0170,
         -4.7000,  -3.8170,  -6.5230, -11.6000,  -1.4800,  -5.8900,  -3.0820,
         -1.4580,   1.1440,  -3.9990,  -1.4700,  -3.1000,  -4.6500,  -2.4000,
         -0.6670,   0.1060,  -3.8930,  -4.2100,  -1.6400,   0.7000,  -1.5500,
         -4.4500,  -3.1500,  -2.3700,  -0.1400,  -4.3700,  -3.6700,  -4.2300,
         -2.2660,  -4.7490,  -5.0300,  -2.0000,  -3.0900,  -2.1110,  -2.4900,
         -4.8900,  -2.5700,  -6.4900,  -3.2390,  -5.1900,  -1.5000,  -4.7100,
         -5.2700,  -1.0600,  -2.5500,  -4.6300,  -2.5900,  -3.0300,  -1.9600,
          0.4500,  -2.3900,  -1.3400,  -3.7800,  -4.2100,  -3.6400,  -8.4000,
         -2.1000,  -0.2800,  -2.2400,  -1.7420,  -2.1200,  -4.26


RMSE, MAE = 2.10±0.006, 1.67±0.004
[[-2.8336792]
 [-2.853123 ]
 [-2.8794162]
 [-2.8364966]
 [-2.8079185]
 [-2.8300369]
 [-2.8059177]
 [-2.8378677]
 [-2.8374112]
 [-2.8629813]
 [-2.8664596]
 [-2.833024 ]
 [-2.8237448]
 [-2.8134615]
 [-2.815373 ]
 [-2.847785 ]
 [-2.8103306]
 [-2.8268316]
 [-2.8218355]
 [-2.810122 ]
 [-2.8396363]
 [-2.846435 ]
 [-2.8925176]
 [-2.8349934]
 [-2.827401 ]
 [-2.8539357]
 [-2.853913 ]
 [-2.8656428]
 [-2.8275073]
 [-2.8498447]
 [-2.8375607]
 [-2.8925176]
 [-2.8454742]
 [-2.8720813]
 [-2.8208725]
 [-2.8240223]
 [-2.8279939]
 [-2.8475516]
 [-2.83771  ]
 [-2.8526626]
 [-2.8484628]
 [-2.8353796]
 [-2.8186455]
 [-2.8147757]
 [-2.8486302]
 [-2.8447042]
 [-2.8518093]
 [-2.8021946]
 [-2.803871 ]
 [-2.8520572]
 [-2.837915 ]
 [-2.834835 ]
 [-2.817112 ]
 [-2.83989  ]
 [-2.855029 ]
 [-2.8337092]
 [-2.8306136]
 [-2.8436823]
 [-2.8620932]
 [-2.8662229]
 [-2.8573976]
 [-2.81973  ]
 [-2.826407 ]
 [-2.8724298]
 [-2.831632 ]
 [-2.8349268]
 [-2.811931 ]
 [-2.8690739]
 [-2.8506515